In [1]:
import pandas as pd
import tqdm
import datetime
from datetime import timedelta
import os

# Učitavanje originalnih podataka 

In [2]:
dataset_path = '../data/raw/export_df.parquet'

In [3]:
data = pd.read_parquet(dataset_path)
data.head()


,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id
0,0,2015-07-01,2015-01-21,2015-07-01,None,2,0.0,PRT,0,0,100.000000,Check-Out,1313223,1077152
1,0,2015-07-01,2015-06-10,2015-07-01,None,2,0.0,PRT,0,0,100.000000,Check-Out,1313224,1017906
2,0,2015-07-01,2015-05-13,2015-07-02,None,1,0.0,GBR,0,1,64.991345,Check-Out,1313225,1039896
3,0,2015-07-01,2014-05-30,2015-07-02,None,1,0.0,GBR,1,1,74.368897,Check-Out,1313226,1008245
4,0,2015-07-01,2014-07-06,2015-07-03,None,2,0.0,GBR,2,1,130.973278,Check-Out,1313227,1093703


#### Uklanjanje zapisa (redaka) gdje je zemlja gosta 0 

In [4]:
data = data[~(data['zemlja_gosta']=='0')]

#### Dodavanje stupca  ukupno_gostiju ; broj djece za svaki redak (djeca + odrasli)

In [5]:
data['ukupno_gostiju'] = data['broj_odraslih_gostiju'] + data['broj_djece_gostiju']
data['ukupno_gostiju'] = data['ukupno_gostiju'].astype('int')

#### Uklanjanje rezervacija u kojima nema gostiju, odnosno ukupan je broj gostiju 0

NOTE: Zašto ove retke uklanjamo sada, a ne nakon primjene statističke ili metode strojnog učenja? Jer ovdje govorimo o retcima u kojima je broj gostiju nula, što znači kako su ti podaci nevaljani: označen je dolazak gosta, odnosno očekujemo da je gost došao, ali je zbog nekog razloga krivo postavljen broj gostiju te možemo reći da su ovdje krivo uneseni podaci

In [6]:
data = data[data['ukupno_gostiju'] != 0]

#### Dodavanje stupca duljina boravka

In [7]:
data['duljina_boravka'] = data['datum_odjave'] - data['datum_dolaska']
data['duljina_boravka'] = data['duljina_boravka'].dt.days

#### Uklanjanje redaka gdje je duljina boravka 0

In [8]:
data = data[data['duljina_boravka']>0]

#### Dodavanje stupca raspon_dolazak_rezervacija

In [9]:
data['raspon_dolazak_rezervacija'] = (data['datum_dolaska'] - data['datum_kreiranja_rezervacije']).dt.days

#### Uklanjanje (u zaseban df) radaka s otkazanim rezervacijama

In [10]:
data_canceled = data[data['datum_otkazivanja_rezervacije'].notna()]
data_canceled = data_canceled.reset_index(drop=True)
data = data[~data['datum_otkazivanja_rezervacije'].notna()]
data = data.reset_index(drop=True)
data.head(5)

,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,2015-05-13,2015-07-02,None,1,0.0,GBR,0,1,64.991345,Check-Out,1313225,1039896,1,1,49
1,0,2015-07-01,2014-05-30,2015-07-02,None,1,0.0,GBR,1,1,74.368897,Check-Out,1313226,1008245,1,1,397
2,0,2015-07-01,2014-07-06,2015-07-03,None,2,0.0,GBR,2,1,130.973278,Check-Out,1313227,1093703,2,2,360
3,0,2015-07-01,2015-04-21,2015-07-03,None,2,0.0,GBR,2,1,143.304560,Check-Out,1313228,1044263,2,2,71
4,0,2015-07-01,2015-06-25,2015-07-03,None,2,0.0,PRT,0,0,161.502814,Check-Out,1313229,1022260,2,2,6


In [11]:
data_canceled.head()

,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,2015-06-30,2015-07-04,2015-05-06,2,0.0,PRT,2,1,61.437192,Canceled,1313231,1075633,2,3,1
1,0,2015-07-01,2015-06-28,2015-07-04,2015-04-22,2,0.0,PRT,2,2,150.365262,Canceled,1313232,1056139,2,3,3
2,0,2015-07-01,2015-04-17,2015-07-05,2015-06-23,2,0.0,PRT,2,3,142.635814,Canceled,1313233,1020092,2,4,75
3,0,2015-07-01,2015-01-04,2015-07-08,2015-05-11,2,0.0,PRT,2,3,113.277032,Canceled,1313250,1018277,2,7,178
4,0,2015-07-01,2015-01-01,2015-07-11,2015-05-29,2,0.0,PRT,0,3,105.416363,Canceled,1313255,1035627,2,10,181


####  Uklanjanje stupaca: 
   - datum otkazivanja rezervacije (ovdje imamo samo neotkazane rezervacije)
   - datum odjave : jer je autokoreliran s stupcima datum dolaska i duljina boravka
   - broj_odraslih_gostiju	te broj_djece_gostiju uklanjamo kako smo dodali stupac ukupan broj gostiju
   - status rezervacije (sve su vrijednosti iste iste - 'Check-Out')
   - rezervacija_id - od 74553 zapisa imamo i 74553 unikatne vrijednosti identifikatora
    (NE ODBACUJEMO gost_id jer od 74553 zapisaimamo 69764 gosta, što znači da imamo ponavljajućih gostiju)
   - datum_kreiranja_rezervacije - dodan je stupac raspon_dolazak_rezervacija koji označava razliku dana dolaska i kreiranja rezervacije 
   

In [12]:
data = data.drop(columns = ['datum_otkazivanja_rezervacije','datum_odjave','broj_odraslih_gostiju','broj_djece_gostiju','status_rezervacije','rezervacija_id','datum_kreiranja_rezervacije'],axis=1)
data_canceled = data_canceled.drop(columns = ['datum_odjave','broj_odraslih_gostiju','broj_djece_gostiju','status_rezervacije','rezervacija_id','datum_kreiranja_rezervacije'],axis=1)
data.head()

,hotel_id,datum_dolaska,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,GBR,0,1,64.991345,1039896,1,1,49
1,0,2015-07-01,GBR,1,1,74.368897,1008245,1,1,397
2,0,2015-07-01,GBR,2,1,130.973278,1093703,2,2,360
3,0,2015-07-01,GBR,2,1,143.304560,1044263,2,2,71
4,0,2015-07-01,PRT,0,0,161.502814,1022260,2,2,6


#### Odvajanje na hotel 0 (Resort) te na hotel 1 (City)

In [13]:
dataResort = data[data['hotel_id'] == 0]
dataCity = data[data['hotel_id'] == 1]

assert len(dataResort) + len(dataCity) == len(data)

# Stvaranje podatkovnih okvira na dnevnoj, tjednoj, mjesečnoj i godišnjoj razini za oba hotela

## Hotel 1 (City Hotel)

### Dnevna razina

In [14]:
daily_guest_counts_city_hotel = {}
for index, row in dataCity.iterrows():
    
    arrival_date = row['datum_dolaska']
    length_of_stay = row['duljina_boravka']
    number_of_guests = row['ukupno_gostiju']
    
    for day in range(length_of_stay + 1):
        current_day = arrival_date + timedelta(days=day)

        if current_day not in daily_guest_counts_city_hotel: #init
            daily_guest_counts_city_hotel[current_day] = 0

        daily_guest_counts_city_hotel[current_day] += number_of_guests

NumberOfGuestsDailyCity = pd.DataFrame(list(daily_guest_counts_city_hotel.items()), columns=['datum', 'ukupno_gostiju'])

In [15]:
NumberOfGuestsDailyCity['datum'] = pd.to_datetime(NumberOfGuestsDailyCity['datum'])
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.sort_values(by='datum')
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.set_index('datum')
NumberOfGuestsDailyCity.head(5)

,ukupno_gostiju
datum,
2015-01-01,10
2015-01-02,16
2015-01-03,27
2015-01-04,27
2015-01-05,28


In [16]:
NumberOfArrivalsDailyCity = dataCity.groupby('datum_dolaska').size()
NumberOfArrivalsDailyCity = NumberOfArrivalsDailyCity.resample('D').sum().to_frame(name='broj_dolazaka')
NumberOfArrivalsDailyCity.head(5)

,broj_dolazaka
datum_dolaska,
2015-01-01,6
2015-01-02,3
2015-01-03,5
2015-01-04,5
2015-01-05,2


In [17]:
NumberOfNewGuestsDailyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum()
NumberOfNewGuestsDailyCity = NumberOfNewGuestsDailyCity.resample('d').sum().to_frame()
NumberOfNewGuestsDailyCity.head(5)

,ukupno_gostiju
datum_dolaska,
2015-01-01,10
2015-01-02,6
2015-01-03,11
2015-01-04,8
2015-01-05,4


### Tjedna razina

KOMENTAR 
    - modeliranje broja gostiju an tjednoj, mjesečnoj i godišnjoj bazi ima sljedeće probleme:
        - uzmemo li na primjer dvije osobe koje borave u istom hotelu u istom tjednu, gledanjem broja gostiju na                tjednoj bazi zanemarujemo razliku u slučajevima: osobe borave na različite dane u tjednu u hotelu (ukupan            broj gostiju +2), osobe borave na identične dane u tjednu u hotelu (opet ukupan broj +2) te osobe samo               djelomično borave na iste dane u hotelu (ukupan broj +2)
        - prethodno + osoba boravi u hotelu tokom dva ili više tjedana

Trenutni kod prolazi kroz svaku rezervaciju te gleda vrijeme dolaska i odlaska te se broj gostiju uvećava za broj gostiju rezervacije za tjedan/mjesec dolaska, boravka ili odlaksa 
Time dobivamo informaciju o broju gostiju u nekom tjednu/mjesecu u apsolutnom iznosu, odnosno ukupan broj gostiju u tom periodu. Isto tako imamo i podatkovni okvir koji čuva infomraciju o broju "novih" gostiju, odnosno promatra se samo dan/tjedan/mjesec dolaska te se na taj način modelira broj doalazaka u nekom vremenskom intervalu (tu se s druge strane gubi informacija o duljini boravka gostiju).
Moguće je rekreirat maximalnu dnevnu okupiranost na tjednoj i mjesečnoj razini (maksimalni dnevni iznos broja gostiju u tjednu/mjesecu) te na taj način modelirati broj gostiju na tjednoj i mjesečnoj razini.
        
        

In [18]:
weekly_guest_counts_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']

    arrival_week_start = arrival_date - timedelta(days=arrival_date.weekday())

    if arrival_week_start.year == 2014:
        arrival_week_start = pd.Timestamp(year=2015, month=1, day=1)

    departure_week_start = departure_date - timedelta(days=departure_date.weekday())
    
    if departure_week_start.year == 2014:
        departure_week_start = pd.Timestamp(year=2015, month=1, day=1)
        
    current_week_start = arrival_week_start
    while current_week_start <= departure_week_start:
        if current_week_start not in weekly_guest_counts_city_hotel:
            weekly_guest_counts_city_hotel[current_week_start] = 0

        weekly_guest_counts_city_hotel[current_week_start] += number_of_guests

        current_week_start += timedelta(days=7)
NumberOfGuestsWeeklyCity = pd.DataFrame(list(weekly_guest_counts_city_hotel.items()), columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfGuestsWeeklyCity['datum_dolaska'] = pd.to_datetime(NumberOfGuestsWeeklyCity['datum_dolaska'])

In [19]:
NumberOfGuestsWeeklyCity = NumberOfGuestsWeeklyCity.sort_values(by='datum_dolaska')
NumberOfGuestsWeeklyCity = NumberOfGuestsWeeklyCity.set_index('datum_dolaska')

In [20]:
NumberOfGuestsWeeklyCity.head()

,ukupno_gostiju
datum_dolaska,
2015-01-01,35
2015-01-05,42
2015-01-12,70
2015-01-19,78
2015-01-26,52


In [21]:
if NumberOfArrivalsDailyCity.index.min() == pd.Timestamp('2015-01-01'):
    NumberOfArrivalsWeeklyCity = NumberOfArrivalsDailyCity.resample('W').sum()
    NumberOfArrivalsWeeklyCity.index = NumberOfArrivalsWeeklyCity.index - pd.DateOffset(days=6)
else:
    NumberOfArrivalsWeeklyCity = NumberOfArrivalsDailyCity.resample('W-Mon').sum()

NumberOfArrivalsWeeklyCity = NumberOfArrivalsWeeklyCity.reset_index()
NumberOfArrivalsWeeklyCity.rename(columns={'index': 'datum_dolaska'}, inplace=True)
NumberOfArrivalsWeeklyCity['datum_dolaska'] = NumberOfArrivalsWeeklyCity['datum_dolaska'].apply(lambda x: pd.Timestamp('2015-01-01') if x.year == 2014 else x)
NumberOfArrivalsWeeklyCity = NumberOfArrivalsWeeklyCity.set_index('datum_dolaska')



In [22]:
NumberOfArrivalsWeeklyCity.head()


,broj_dolazaka
datum_dolaska,
2015-01-01,19
2015-01-05,23
2015-01-12,33
2015-01-19,32
2015-01-26,15


In [23]:
if NumberOfNewGuestsDailyCity.index.min() == pd.Timestamp('2015-01-01'):
    NumberOfNewGuestsWeeklyCity = NumberOfNewGuestsDailyCity.resample('W').sum()
    NumberOfNewGuestsWeeklyCity.index = NumberOfNewGuestsWeeklyCity.index - pd.DateOffset(days=6)
else:
    NumberOfNewGuestsWeeklyCity = NumberOfNewGuestsDailyCity.resample('W-Mon').sum()

NumberOfNewGuestsWeeklyCity = NumberOfNewGuestsWeeklyCity.reset_index()
NumberOfNewGuestsWeeklyCity.rename(columns={'index': 'datum_dolaska'}, inplace=True)

NumberOfNewGuestsWeeklyCity['datum_dolaska'] = NumberOfNewGuestsWeeklyCity['datum_dolaska'].apply(lambda x: pd.Timestamp('2015-01-01') if x.year == 2014 else x)
NumberOfNewGuestsWeeklyCity = NumberOfNewGuestsWeeklyCity.set_index('datum_dolaska')

In [24]:
NumberOfNewGuestsWeeklyCity.head()

,ukupno_gostiju
datum_dolaska,
2015-01-01,35
2015-01-05,42
2015-01-12,54
2015-01-19,48
2015-01-26,27


### Mjesečna razina

In [25]:
monthly_guest_counts_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']

    current_month_start = arrival_date.replace(day=1)

    while current_month_start <= departure_date.replace(day=1):
        #print(f"Arr {arrival_date}, curr {current_month_start}, dep {departure_date}")
        if current_month_start not in monthly_guest_counts_city_hotel:
            monthly_guest_counts_city_hotel[current_month_start] = 0

        monthly_guest_counts_city_hotel[current_month_start] += number_of_guests

        if current_month_start.month == 12: 
            current_month_start = current_month_start.replace(year=current_month_start.year + 1, month=1, day=1)
        else:
            current_month_start = current_month_start.replace(month=current_month_start.month + 1, day=1)

monthly_guest_data = [(date, guests) for date, guests in monthly_guest_counts_city_hotel.items()]
NumberOfGuestsMonthlyCity = pd.DataFrame(monthly_guest_data, columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfGuestsMonthlyCity = NumberOfGuestsMonthlyCity.sort_values(by='datum_dolaska')
NumberOfGuestsMonthlyCity = NumberOfGuestsMonthlyCity.set_index('datum_dolaska')


In [26]:
NumberOfGuestsMonthlyCity.head(5)

,ukupno_gostiju
datum_dolaska,
2015-01-01,206
2015-02-01,9
2015-05-01,1067
2015-06-01,62
2015-07-01,442


In [27]:
NumberOfArrivalsMonthlyCity = NumberOfArrivalsDailyCity.resample('MS').sum()
NumberOfArrivalsMonthlyCity.head()

,broj_dolazaka
datum_dolaska,
2015-01-01,122
2015-02-01,0
2015-03-01,0
2015-04-01,0
2015-05-01,574


In [28]:
NumberOfNewGuestsMonthlyCity = NumberOfNewGuestsDailyCity.resample('MS').sum()
NumberOfNewGuestsMonthlyCity.head()

,ukupno_gostiju
datum_dolaska,
2015-01-01,206
2015-02-01,0
2015-03-01,0
2015-04-01,0
2015-05-01,1067


### Godišnja razina

In [29]:
yearly_guest_counts_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']

    current_year_start = arrival_date.replace(month=1, day=1)

    while current_year_start.year <= departure_date.year:
        if current_year_start not in yearly_guest_counts_city_hotel:
            yearly_guest_counts_city_hotel[current_year_start] = 0

        yearly_guest_counts_city_hotel[current_year_start] += number_of_guests

        current_year_start = current_year_start.replace(year=current_year_start.year + 1, month=1, day=1)

yearly_guest_data = [(date, guests) for date, guests in yearly_guest_counts_city_hotel.items()]
NumberOfGuestsYearlyCity = pd.DataFrame(yearly_guest_data, columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfGuestsYearlyCity = NumberOfGuestsYearlyCity.sort_values(by='datum_dolaska')


In [30]:
NumberOfGuestsYearlyCity = NumberOfGuestsYearlyCity.set_index('datum_dolaska')
NumberOfGuestsYearlyCity.head(5)

,ukupno_gostiju
datum_dolaska,
2015-01-01,13642
2016-01-01,44621
2017-01-01,32100
2018-01-01,360


In [31]:
NumberOfArrivalsYearlyCity =NumberOfArrivalsDailyCity.resample('YS').sum()

In [32]:
NumberOfArrivalsYearlyCity

,broj_dolazaka
datum_dolaska,
2015-01-01,7598
2016-01-01,22493
2017-01-01,15705


In [33]:
NumberOfNewGuestsYearlyCity = NumberOfNewGuestsDailyCity.resample('YS').sum()

In [34]:
NumberOfNewGuestsYearlyCity

,ukupno_gostiju
datum_dolaska,
2015-01-01,13642
2016-01-01,44131
2017-01-01,31170


## Hotel 0 (Resort Hotel)

### Dnevna razina

In [35]:
daily_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    
    arrival_date = row['datum_dolaska']
    length_of_stay = row['duljina_boravka']
    number_of_guests = row['ukupno_gostiju']
    
    for day in range(length_of_stay + 1):
        current_day = arrival_date + timedelta(days=day)

        if current_day not in daily_guest_counts_resort_hotel: #init
            daily_guest_counts_resort_hotel[current_day] = 0

        daily_guest_counts_resort_hotel[current_day] += number_of_guests

NumberOfGuestsDailyResort = pd.DataFrame(list(daily_guest_counts_resort_hotel.items()), columns=['datum', 'ukupno_gostiju'])

In [36]:
NumberOfGuestsDailyResort['datum'] = pd.to_datetime(NumberOfGuestsDailyResort['datum'])
NumberOfGuestsDailyResort = NumberOfGuestsDailyResort.sort_values(by='datum')
NumberOfGuestsDailyResort = NumberOfGuestsDailyResort.set_index('datum')
NumberOfGuestsDailyResort.head(5)

,ukupno_gostiju
datum,
2015-07-01,63
2015-07-02,135
2015-07-03,187
2015-07-04,245
2015-07-05,307


In [37]:
NumberOfArrivalsDailyResort = dataResort.groupby('datum_dolaska').size()
NumberOfArrivalsDailyResort = NumberOfArrivalsDailyResort.resample('D').sum().to_frame(name='broj_dolazaka')
NumberOfArrivalsDailyResort.head(5)

,broj_dolazaka
datum_dolaska,
2015-07-01,32
2015-07-02,35
2015-07-03,27
2015-07-04,36
2015-07-05,37


In [38]:
NumberOfNewGuestsDailyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfNewGuestsDailyResort = NumberOfNewGuestsDailyResort.resample('d').sum().to_frame()
NumberOfNewGuestsDailyResort.head(5)

,ukupno_gostiju
datum_dolaska,
2015-07-01,63
2015-07-02,72
2015-07-03,59
2015-07-04,80
2015-07-05,80


### Tjedna razina

In [39]:
weekly_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']

    arrival_week_start = arrival_date - timedelta(days=arrival_date.weekday())

    if arrival_week_start.year == 2014:
        arrival_week_start = pd.Timestamp(year=2015, month=1, day=1)

    departure_week_start = departure_date - timedelta(days=departure_date.weekday())
    
    if departure_week_start.year == 2014:
        departure_week_start = pd.Timestamp(year=2015, month=1, day=1)
        
    current_week_start = arrival_week_start
    while current_week_start <= departure_week_start:
        if current_week_start not in weekly_guest_counts_resort_hotel:
            weekly_guest_counts_resort_hotel[current_week_start] = 0

        weekly_guest_counts_resort_hotel[current_week_start] += number_of_guests

        current_week_start += timedelta(days=7)

In [40]:
NumberOfGuestsWeeklyResort  = pd.DataFrame(list(weekly_guest_counts_resort_hotel.items()), columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfGuestsWeeklyResort ['datum_dolaska'] = pd.to_datetime(NumberOfGuestsWeeklyResort['datum_dolaska'])
NumberOfGuestsWeeklyResort = NumberOfGuestsWeeklyResort.sort_values(by='datum_dolaska')
NumberOfGuestsWeeklyResort  = NumberOfGuestsWeeklyResort.set_index('datum_dolaska')
NumberOfGuestsWeeklyResort.head(5)

,ukupno_gostiju
datum_dolaska,
2015-06-29,354
2015-07-06,736
2015-07-13,918
2015-07-20,895
2015-07-27,933


In [41]:
week_starts = NumberOfGuestsWeeklyResort.index
weekly_arrivals = []
for i in range(len(week_starts) - 1):
    week_start = week_starts[i]
    next_week_start = week_starts[i + 1]
    weekly_total = NumberOfArrivalsDailyResort[week_start:next_week_start - pd.DateOffset(days=1)]['broj_dolazaka'].sum()
    weekly_arrivals.append((week_start, weekly_total))
NumberOfArrivalsWeeklyResort = pd.DataFrame(weekly_arrivals, columns=['datum_dolaska', 'broj_dolazaka'])
NumberOfArrivalsWeeklyResort.set_index('datum_dolaska', inplace=True)

In [42]:
NumberOfArrivalsWeeklyResort.head()

,broj_dolazaka
datum_dolaska,
2015-06-29,167
2015-07-06,215
2015-07-13,249
2015-07-20,231
2015-07-27,245


In [43]:
week_starts = NumberOfGuestsWeeklyResort.index.tolist()
weekly_new_guests_data = []

for i in range(len(week_starts) - 1):
    week_start = week_starts[i]
    next_week_start = week_starts[i + 1]
    weekly_total = NumberOfNewGuestsDailyResort[week_start:next_week_start - pd.Timedelta(days=1)].sum()
    weekly_new_guests_data.append([week_start, weekly_total['ukupno_gostiju']])  

NumberOfNewGuestsWeeklyResort = pd.DataFrame(weekly_new_guests_data, columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfNewGuestsWeeklyResort.set_index('datum_dolaska', inplace=True)

In [44]:
NumberOfNewGuestsWeeklyResort.head()

,ukupno_gostiju
datum_dolaska,
2015-06-29,354
2015-07-06,483
2015-07-13,535
2015-07-20,521
2015-07-27,547


### Mjesečna razina

In [45]:
monthly_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']

    current_month_start = arrival_date.replace(day=1)

    while current_month_start <= departure_date.replace(day=1):
        if current_month_start not in monthly_guest_counts_resort_hotel:
            monthly_guest_counts_resort_hotel[current_month_start] = 0

        monthly_guest_counts_resort_hotel[current_month_start] += number_of_guests

        if current_month_start.month == 12: 
            current_month_start = current_month_start.replace(year=current_month_start.year + 1, month=1, day=1)
        else:
            current_month_start = current_month_start.replace(month=current_month_start.month + 1, day=1)

monthly_guest_data_resort = [(date, guests) for date, guests in monthly_guest_counts_resort_hotel.items()]
NumberOfGuestsMonthlyResort = pd.DataFrame(monthly_guest_data_resort, columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfGuestsMonthlyResort = NumberOfGuestsMonthlyResort.sort_values(by='datum_dolaska')
NumberOfGuestsMonthlyResort = NumberOfGuestsMonthlyResort.set_index('datum_dolaska')

In [46]:
NumberOfGuestsMonthlyResort.head(5)

,ukupno_gostiju
datum_dolaska,
2015-07-01,2263
2015-08-01,2678
2015-09-01,2362
2015-10-01,2419
2015-11-01,1722


In [47]:
NumberOfArrivalsMonthlyResort = NumberOfArrivalsDailyResort.resample('MS').sum()

In [48]:
NumberOfArrivalsMonthlyResort.head()

,broj_dolazaka
datum_dolaska,
2015-07-01,1028
2015-08-01,1014
2015-09-01,1017
2015-10-01,1115
2015-11-01,884


In [49]:
NumberOfNewGuestsMonthlyResort = NumberOfNewGuestsDailyResort.resample('MS').sum()

In [50]:
NumberOfNewGuestsMonthlyResort.head()

,ukupno_gostiju
datum_dolaska,
2015-07-01,2263
2015-08-01,2278
2015-09-01,2006
2015-10-01,2072
2015-11-01,1501


### Godišnja razina

In [51]:
yearly_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']

    current_year_start = arrival_date.replace(month=1, day=1)

    while current_year_start.year <= departure_date.year:
        if current_year_start not in yearly_guest_counts_resort_hotel:
            yearly_guest_counts_resort_hotel[current_year_start] = 0

        yearly_guest_counts_resort_hotel[current_year_start] += number_of_guests

        current_year_start = current_year_start.replace(year=current_year_start.year + 1, month=1, day=1)

yearly_guest_data_resort = [(date, guests) for date, guests in yearly_guest_counts_resort_hotel.items()]
NumberOfGuestsYearlyResort = pd.DataFrame(yearly_guest_data_resort, columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfGuestsYearlyResort = NumberOfGuestsYearlyResort.sort_values(by='datum_dolaska')
NumberOfGuestsYearlyResort = NumberOfGuestsYearlyResort.set_index('datum_dolaska')


In [52]:
NumberOfGuestsYearlyResort.head(5)

,ukupno_gostiju
datum_dolaska,
2015-01-01,11858
2016-01-01,26068
2017-01-01,18056


In [53]:
NumberOfArrivalsYearlyResort = NumberOfArrivalsDailyResort.resample('YS').sum()

In [54]:
NumberOfArrivalsYearlyResort

,broj_dolazaka
datum_dolaska,
2015-01-01,5970
2016-01-01,13183
2017-01-01,8987


In [55]:
NumberOfNewGuestsYearlyResort = NumberOfNewGuestsDailyResort.resample('YS').sum()

In [56]:
NumberOfNewGuestsYearlyResort

,ukupno_gostiju
datum_dolaska,
2015-01-01,11858
2016-01-01,25708
2017-01-01,17715


### Spremanje podatakovnih okvira (interim za sada)

In [57]:
DATA_PATH = '../data/interim'
DATA_HOTEL0_BASE_PATH = '../data/interim/hotel_0'
DATA_HOTEL1_BASE_PATH = '../data/interim/hotel_1'

In [58]:
data.to_parquet(os.path.join(DATA_PATH,"data_no_cancellations.parquet"))
data_canceled.to_parquet(os.path.join(DATA_PATH,"data_cancellations.parquet"))
dataCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"hotel0cleaned.parquet"))
dataResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"hotel1cleaned.parquet"))



In [59]:
NumberOfGuestsDailyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_guests.parquet"))
NumberOfArrivalsDailyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_arrivals.parquet")) 
NumberOfNewGuestsDailyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_new_guests.parquet"))
NumberOfGuestsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_guests.parquet")) 
NumberOfArrivalsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_arrivals.parquet")) 
NumberOfNewGuestsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_new_guests.parquet"))
NumberOfGuestsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_guests.parquet")) 
NumberOfArrivalsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_arrivals.parquet"))
NumberOfNewGuestsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_new_guests.parquet"))
NumberOfGuestsYearlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_guests.parquet"))
NumberOfArrivalsYearlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_arrivals.parquet")) 
NumberOfNewGuestsYearlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_new_guests.parquet"))

In [60]:
NumberOfGuestsDailyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_guests.parquet"))
NumberOfArrivalsDailyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_arrivals.parquet"))
NumberOfNewGuestsDailyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_new_guests.parquet"))
NumberOfGuestsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_guests.parquet"))
NumberOfArrivalsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_arrivals.parquet"))
NumberOfNewGuestsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_new_guests.parquet"))
NumberOfGuestsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_guests.parquet"))
NumberOfArrivalsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_arrivals.parquet"))
NumberOfNewGuestsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_new_guests.parquet"))
NumberOfGuestsYearlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_guests.parquet"))
NumberOfArrivalsYearlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_arrivals.parquet"))
NumberOfNewGuestsYearlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_new_guests.parquet"))